# Examine a single word  


In [ ]:
import os
os.chdir('/home/jupyter/triangle_model')
import troubleshooting 
from ipywidgets import interact

# Intactive plot 

In [ ]:
# code_name = 'OP'
# code_name = 'Refrac3_local'
# code_name = 'Refrac3_after6'
# code_name = 'Refrac3_after6_nodc'
# code_name = 'Refrac3_after6_init_act0'
# code_name = 'Refrac3_after6_informative'
# code_name = 'OS_ff'
code_name = 'triangle_nodc_osop'
testset_name = 'train_r100'
d = troubleshooting.Diagnosis(code_name)
d.eval(testset_name, task='triangle', epoch=200)

@interact(
    sel_word=d.testset_package['item'], 
    layer=['pho', 'sem'], 
    task=['exp_os_ff', 'triangle', 'ort_pho', 'exp_osp', 'ort_sem', 'exp_ops'], 
    epoch=(10, d.cfg.total_number_of_epoch + 1, d.cfg.save_freq)
    )
def interactive_plot(sel_word, layer, task, epoch):
    d = troubleshooting.Diagnosis(code_name)
    d.eval(testset_name, task=task, epoch=epoch)
    d.set_target_word(sel_word)
    print(f"Output phoneme over timeticks: {d.list_output_phoneme}")
    return d.plot_one_layer(layer)

- In an intact OP model, ignoring the non informative ticks, the general pattern

## We found a very weird SS-1 having a very high input in off nodes as compared to on nodes
- This pattern is also exist in PHO layer
Consider PHO layer in a OP model

1) Tick 1 is non-informative (No information from O had propagated to P yet) --> tick 1 is fixed across words
2) All input is coming from Act P at 0 (=0.5) matmul w_pp. This fixed value is summarized in below density plot
3) Functionally it serve as the knowledge (obtain from pretraining) embedded in w_pp, reflecting some degree of direct (1-to-1) coherence coactivation in the phonological layer
4) Put in the more general context, bias, PP, and CP reflects probably(?) reflects -> independent activation likelihood in each node, immediate coherence coactivation (1-to-1), and slightly more complex (because it goes through a hidden layer) (but compressed) coherence coactivation???
5) Their relative strength is summarized in below density plot



### Tick 1 input density at SEM

In [ ]:
w_hos_hs = d.get_weight(name="w_hos_hs")
w_ss = d.get_weight(name="w_ss")
bias_s = d.get_weight(name="bias_s")
w_cs = d.get_weight(name="w_cs")

data = {
    'ss1': 0.5 * np.sum(w_ss, axis=0), # Lazy matmul. 
    'cs1': 0.5 * np.sum(w_cs, axis=0),
    'os1': 0.5 * np.sum(w_hos_hs, axis=0),
    'bias1': bias_s
}

df = pd.DataFrame.from_dict(data)

fig, ax = plt.subplots(1, 4, figsize=(15,6))
for i, k in enumerate(data.keys()):
    df[k].plot.density(ax=ax[i], title=k)
    


### Tick 1 input density at PHO

In [ ]:
w_hop_hp = d.get_weight(name="w_hop_hp")
w_pp = d.get_weight(name="w_pp")
bias_p = d.get_weight(name="bias_p")
w_cp = d.get_weight(name="w_cp")

data = {
    'pp1': 0.5 * np.sum(w_pp, axis=0), # Lazy matmul. 
    'cp1': 0.5 * np.sum(w_cp, axis=0),
    'op1': 0.5 * np.sum(w_hop_hp, axis=0),
    'bias1': bias_p
}

df = pd.DataFrame.from_dict(data)

fig, ax = plt.subplots(1, 4, figsize=(15,6))
for i, k in enumerate(data.keys()):
    df[k].plot.density(ax=ax[i], title=k)
    
